# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [49]:
import pandas as pd
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
import statsmodels.api as sm
import numpy as np
from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor

In [3]:
df = pd.read_csv('previsao_de_renda.csv', index_col=[0]).drop(['id_cliente', 'data_ref'], axis=1)
df.dropna(inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12427 entries, 0 to 14999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   12427 non-null  object 
 1   posse_de_veiculo       12427 non-null  bool   
 2   posse_de_imovel        12427 non-null  bool   
 3   qtd_filhos             12427 non-null  int64  
 4   tipo_renda             12427 non-null  object 
 5   educacao               12427 non-null  object 
 6   estado_civil           12427 non-null  object 
 7   tipo_residencia        12427 non-null  object 
 8   idade                  12427 non-null  int64  
 9   tempo_emprego          12427 non-null  float64
 10  qt_pessoas_residencia  12427 non-null  float64
 11  renda                  12427 non-null  float64
dtypes: bool(2), float64(3), int64(2), object(5)
memory usage: 1.1+ MB


1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [5]:
# Separando a base entre treino e teste
X_train, X_test = train_test_split(df, test_size=0.25)

In [6]:
# Função para criar os modelos
def create_model(alpha, reg_type, data):
    model = 'np.log(renda) ~ C(sexo) + C(posse_de_veiculo) + C(posse_de_imovel) + C(tipo_renda) + C(educacao) + C(estado_civil) + C(tipo_residencia) + qt_pessoas_residencia + qtd_filhos + tempo_emprego'
    model = smf.ols(model, data)
    reg = model.fit_regularized(method='elastic_net', L1_wt=reg_type, alpha=alpha, refit=True)
    return reg

In [7]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
rsquareds = []

In [8]:
# Regularização ridge - Avaliando o melhor alpha

for alpha in alphas:
  reg = create_model(alpha, 0.01, X_test)
  rsquareds.append({alpha: reg.rsquared})

In [9]:
rsquareds
# O alpha que retornou o melhor R² foi o alpha de valor 0

[{0: 0.3592085250073409},
 {0.001: 0.35917334602204687},
 {0.005: 0.3588295201240439},
 {0.01: 0.3581908004960358},
 {0.05: 0.35841275956065666},
 {0.1: 0.35775351688606016}]

In [10]:
modelo_final = create_model(0, 0.01, X_test)
modelo_final.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.359
Model:                            OLS   Adj. R-squared:                  0.355
Method:                 Least Squares   F-statistic:                     75.17
Date:                Thu, 10 Aug 2023   Prob (F-statistic):          5.13e-277
Time:                        20:18:15   Log-Likelihood:                -3341.4
No. Observations:                3107   AIC:                             6731.
Df Residuals:                    3084   BIC:                             6876.
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               6.3184      0.514     12.303      0.000       5.311       7.325
C(sexo)[T.M]                            0.7734      0.029     26.687      0.000       0.717       0.830
C(posse_de_veiculo)[T.True]             0.0507      0.028      1.809      0.071      -0.004       0.106
C(posse_de_imovel)[T.True]              0.1040      0.028      3.763      0.000       0.050       0.158
C(tipo_renda)[T.Bolsista]               0.3061      0.412      0.742      0.458      -0.502       1.115
C(tipo_renda)[T.Empresário]             0.1623      0.030      5.475      0.000       0.104       0.220
C(tipo_renda)[T.Servidor público]       0.0139      0.045      0.309      0.757      -0.074       0.102
C(educacao)[T.Pós graduação]           -0.0255      0.353     -0.072      0.942      -0.718       0.667
C(educacao)[T.Secundário]              -0.1522      0.149     -1.018      0.309      -0.445       0.141
C(educacao)[T.Superior completo]       -0.0498      0.150     -0.332      0.740      -0.344       0.244
C(educacao)[T.Superior incompleto]     -0.1938      0.159     -1.219      0.223      -0.506       0.118
C(estado_civil)[T.Separado]             0.7223      0.244      2.962      0.003       0.244       1.200
C(estado_civil)[T.Solteiro]             0.5398      0.241      2.239      0.025       0.067       1.013
C(estado_civil)[T.União]               -0.0697      0.050     -1.387      0.166      -0.168       0.029
C(estado_civil)[T.Viúvo]                0.6863      0.251      2.731      0.006       0.194       1.179
C(tipo_residencia)[T.Casa]             -0.1343      0.120     -1.118      0.263      -0.370       0.101
C(tipo_residencia)[T.Com os pais]      -0.1507      0.131     -1.147      0.251      -0.408       0.107
C(tipo_residencia)[T.Comunitário]      -0.2492      0.209     -1.193      0.233      -0.659       0.160
C(tipo_residencia)[T.Estúdio]          -0.0835      0.206     -0.405      0.685      -0.488       0.321
C(tipo_residencia)[T.Governamental]    -0.1441      0.138     -1.042      0.297      -0.415       0.127
qt_pessoas_residencia                   0.6253      0.238      2.630      0.009       0.159       1.091
qtd_filhos                             -0.6029      0.238     -2.535      0.011      -1.069      -0.137
tempo_emprego                           0.0637      0.002     32.255      0.000       0.060       0.068
==============================================================================
Omnibus:                        0.209   Durbin-Watson:                   1.955
Prob(Omnibus):                  0.901   Jarque-Bera (JB):                0.165
Skew:                          -0.011   Prob(JB):                        0.921
Kurtosis:                       3.028   Cond. No.              

In [11]:
rsquareds_ = []

In [12]:
# Regularização LASSO - Avaliando o melhor alpha

for alpha in alphas:
  reg = create_model(alpha, 1.0, X_test)
  rsquareds_.append({alpha: reg.rsquared})

In [13]:
rsquareds_

[{0: 0.3592085250073409},
 {0.001: 0.3565597670362646},
 {0.005: 0.34671406982570585},
 {0.01: 0.3463974620191105},
 {0.05: 0.3402329114511926},
 {0.1: 0.3402329114511926}]

In [14]:
# O alpha que retornou o melhor R² foi novamente o alpha de valor 0. Ambos os métodos chegaram no mesmo resultado
modelo_final_2 = create_model(0, 1.0, X_test)
modelo_final_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.359
Model:                            OLS   Adj. R-squared:                  0.355
Method:                 Least Squares   F-statistic:                     75.17
Date:                Thu, 10 Aug 2023   Prob (F-statistic):          5.13e-277
Time:                        20:18:18   Log-Likelihood:                -3341.4
No. Observations:                3107   AIC:                             6731.
Df Residuals:                    3084   BIC:                             6876.
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               6.3184      0.514     12.303      0.000       5.311       7.325
C(sexo)[T.M]                            0.7734      0.029     26.687      0.000       0.717       0.830
C(posse_de_veiculo)[T.True]             0.0507      0.028      1.809      0.071      -0.004       0.106
C(posse_de_imovel)[T.True]              0.1040      0.028      3.763      0.000       0.050       0.158
C(tipo_renda)[T.Bolsista]               0.3061      0.412      0.742      0.458      -0.502       1.115
C(tipo_renda)[T.Empresário]             0.1623      0.030      5.475      0.000       0.104       0.220
C(tipo_renda)[T.Servidor público]       0.0139      0.045      0.309      0.757      -0.074       0.102
C(educacao)[T.Pós graduação]           -0.0255      0.353     -0.072      0.942      -0.718       0.667
C(educacao)[T.Secundário]              -0.1522      0.149     -1.018      0.309      -0.445       0.141
C(educacao)[T.Superior completo]       -0.0498      0.150     -0.332      0.740      -0.344       0.244
C(educacao)[T.Superior incompleto]     -0.1938      0.159     -1.219      0.223      -0.506       0.118
C(estado_civil)[T.Separado]             0.7223      0.244      2.962      0.003       0.244       1.200
C(estado_civil)[T.Solteiro]             0.5398      0.241      2.239      0.025       0.067       1.013
C(estado_civil)[T.União]               -0.0697      0.050     -1.387      0.166      -0.168       0.029
C(estado_civil)[T.Viúvo]                0.6863      0.251      2.731      0.006       0.194       1.179
C(tipo_residencia)[T.Casa]             -0.1343      0.120     -1.118      0.263      -0.370       0.101
C(tipo_residencia)[T.Com os pais]      -0.1507      0.131     -1.147      0.251      -0.408       0.107
C(tipo_residencia)[T.Comunitário]      -0.2492      0.209     -1.193      0.233      -0.659       0.160
C(tipo_residencia)[T.Estúdio]          -0.0835      0.206     -0.405      0.685      -0.488       0.321
C(tipo_residencia)[T.Governamental]    -0.1441      0.138     -1.042      0.297      -0.415       0.127
qt_pessoas_residencia                   0.6253      0.238      2.630      0.009       0.159       1.091
qtd_filhos                             -0.6029      0.238     -2.535      0.011      -1.069      -0.137
tempo_emprego                           0.0637      0.002     32.255      0.000       0.060       0.068
==============================================================================
Omnibus:                        0.209   Durbin-Watson:                   1.955
Prob(Omnibus):                  0.901   Jarque-Bera (JB):                0.165
Skew:                          -0.011   Prob(JB):                        0.921
Kurtosis:                       3.028   Cond. No.              

In [15]:
df_dm = pd.get_dummies(df)
y = df_dm['renda']
X = df_dm.drop('renda', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [20]:
# Regressão stepwise
model = 'np.log(renda) ~ C(sexo) + C(posse_de_veiculo) + C(posse_de_imovel) + C(tipo_renda) + C(educacao) + C(estado_civil) + C(tipo_residencia) + qt_pessoas_residencia + qtd_filhos + tempo_emprego'
st_reg = sm.OLS(y_test.astype(float), X_test.astype(float), model).fit()

In [21]:
st_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.270
Model:                            OLS   Adj. R-squared:                  0.265
Method:                 Least Squares   F-statistic:                     47.59
Date:                Thu, 10 Aug 2023   Prob (F-statistic):          1.81e-190
Time:                        20:21:44   Log-Likelihood:                -31924.
No. Observations:                3107   AIC:                         6.390e+04
Df Residuals:                    3082   BIC:                         6.405e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
posse_de_veiculo                553.0579    276.469      2.000      0.046      10.976    1095.140
posse_de_imovel                 302.1051    273.988      1.103      0.270    -235.112     839.322
qtd_filhos                     -652.0644   1965.759     -0.332      0.740   -4506.394    3202.265
idade                            35.4593     15.857      2.236      0.025       4.368      66.550
tempo_emprego                   509.0392     20.270     25.113      0.000     469.296     548.783
qt_pessoas_residencia           616.8992   1958.502      0.315      0.753   -3223.203    4457.001
sexo_F                        -3045.5169   1241.670     -2.453      0.014   -5480.102    -610.931
sexo_M                         2843.6637   1247.685      2.279      0.023     397.286    5290.042
tipo_renda_Assalariado          335.1878   1030.452      0.325      0.745   -1685.255    2355.630
tipo_renda_Bolsista           -1477.2301   3058.821     -0.483      0.629   -7474.764    4520.303
tipo_renda_Empresário          1108.5275   1037.488      1.068      0.285    -925.710    3142.765
tipo_renda_Pensionista         -938.7888   3470.143     -0.271      0.787   -7742.816    5865.238
tipo_renda_Servidor público     770.4503   1073.685      0.718      0.473   -1334.761    2875.662
educacao_Primário              -570.3222   1347.610     -0.423      0.672   -3212.627    2071.983
educacao_Pós graduação          799.9066   2463.161      0.325      0.745   -4029.697    5629.510
educacao_Secundário            -140.0637    755.897     -0.185      0.853   -1622.178    1342.050
educacao_Superior completo      614.7748    749.022      0.821      0.412    -853.857    2083.407
educacao_Superior incompleto   -906.1487    876.694     -1.034      0.301   -2625.113     812.816
estado_civil_Casado            -290.8925   1636.366     -0.178      0.859   -3499.371    2917.586
estado_civil_Separado           342.4429    648.295      0.528      0.597    -928.692    1613.577
estado_civil_Solteiro           497.2815    528.211      0.941      0.347    -538.400    1532.963
estado_civil_União            -1104.7438   1671.789     -0.661      0.509   -4382.677    2173.189
estado_civil_Viúvo              354.0587    821.261      0.431      0.666   -1256.216    1964.334
tipo_residencia_Aluguel         817.8974   1018.768      0.803      0.422   -1179.635    2815.430
tipo_residencia_Casa           -482.7043    613.996     -0.786      0.432   -1686.587     721.178
tipo_residencia_Com os pais    -515.2064    757.997     -0.680      0.497   -2001.436     971.023
tipo_residencia_Comunitário   -1258.6906   1531.497     -0.822      0.411   -4261.549    1744.168
tipo_residencia_Estúdio        2440.4500   1894.381      1.288      0.198   -1273.928    6154.828
tipo_residencia_Governamental -1203.5992    8

In [18]:
# Os modelos com regressão LASSO e ridge foram melhores

In [52]:
# Rodando uma árvore de regressão
tree =  DecisionTreeRegressor(max_depth= 5, min_samples_split= 10)

In [53]:
dum = pd.get_dummies(df)
X_train, X_test = train_test_split(dum, test_size=0.25)
encoded = preprocessing.LabelEncoder().fit_transform(np.array([1.4,0.4]))

In [59]:
tree.fit(X_train.astype(float), y_train.astype(float))

DecisionTreeRegressor(max_depth=5, min_samples_split=10)

In [61]:
tree.score(X_train, y_train)

0.012339977330830143